## qa셋 전처리 방식별 데이터 정합도/커버율 테스트

In [10]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()

# OpenAI API 키 설정
openai.api_key = os.getenv("OPENAI_API_KEY")

def generate_key_points(text):
    prompt = f"다음 문서에서 핵심적인 기술 포인트 5~10개를 뽑아주세요. 각 포인트는 간결하게 1~2문장으로 작성해 주세요.\n\n{text}"

    # GPT-4o-mini 모델로 핵심 포인트 생성 요청
    response = openai.chat.completions.create(
        model="gpt-4o-mini",  # 모델 이름 (예시: GPT-4o-mini, 원하는 모델을 사용)
        messages=[{"role": "system", "content": "You are a helpful assistant."},  # 시스템 메시지
                  {"role": "user", "content": prompt}],  # 사용자 메시지 (핵심 포인트 요청)
        max_tokens=500,
        temperature=0.
    )

    # 응답에서 핵심 포인트 추출
    key_points = response.choices[0].message.content.strip()

    return key_points

def save_key_points_to_file(key_points, filename="key_points.txt"):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(key_points)

def process_sample_document(file_path):
    # 샘플 문서 파일을 읽어 텍스트 추출
    if not os.path.exists(file_path):
        print(f"파일을 찾을 수 없습니다: {file_path}")
        return

    with open(file_path, 'r', encoding='utf-8') as file:
        sample_text = file.read()

    # 핵심 포인트 생성
    key_points = generate_key_points(sample_text)

    # 생성된 핵심 포인트를 텍스트 파일로 저장
    output_filename = f"key_points_{os.path.basename(file_path)}.txt"
    save_key_points_to_file(key_points, filename=output_filename)

    print(f"핵심 포인트가 '{output_filename}' 파일에 저장되었습니다.")

# 샘플 문서 경로를 입력받아 해당 문서에 대해 처리
sample_document_path = "./people_quickstart_go.txt"  # 여기에 경로를 입력하세요
process_sample_document(sample_document_path)

핵심 포인트가 'key_points_people_quickstart_go.txt.txt' 파일에 저장되었습니다.


In [11]:
from openai import OpenAI
import json
import os
import re
from dotenv import load_dotenv

load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# 파일이 저장된 디렉토리 경로
files_dir = './test'


# 질문-답변 및 출처 생성 함수 (한 번에 처리)
def generate_qa_and_sources(text):
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system",
                    "content": """주어진 People api 문서를 바탕으로 유용한 질문-답변과 해당 답변의 출처 URL을 찾아주세요.

**중요한 제약사항:**
- 문서에 명시된 내용만을 기반으로 질문과 답변을 작성하세요
- 문서에 없는 내용이나 추측, 일반적인 지식을 추가하지 마세요
- 답변은 반드시 문서 내용을 직접 참조해야 합니다
- 확실하지 않은 내용은 포함하지 마세요

**우선적으로 다룰 주제:**
1. 코드 예시와 구현 방법
2. API 사용법과 파라미터 설명
3. 오류 해결 방법과 문제 해결책
4. 설정 방법과 구성 옵션
5. 실제 사용 사례와 예제

**생성 규칙:**
- 문서에서 위 주제에 해당하는 내용이 충분히 있을 때만 질문-답변을 생성하세요
- 내용이 부족하거나 추상적인 설명만 있다면 "생성할 수 없음"이라고 응답하세요
- 최대 10개의 질문-답변을 생성하세요
- 각 질문-답변은 실용적이고 구체적이어야 합니다

**출처 URL 찾기 규칙:**
1. 문서 맨 위에 있는 페이지 URL을 기본으로 포함
2. 답변 내용과 직접 관련된 특정 섹션이나 기능의 URL이 문서 내에 별도로 명시되어 있다면 추가로 포함
3. 내부 링크나 참조 링크가 답변과 관련이 있다면 포함
4. 문서에 실제로 존재하는 URL만 사용하세요

**응답 형식:**
질문1: [문서 내용 기반의 구체적인 질문]
답변1: [문서에 명시된 내용만으로 작성한 답변]
출처1: [문서 내에 실제로 존재하는 URL1, URL2, ...]

질문2: [두 번째 질문]
답변2: [두 번째 답변]
출처2: [URL1, URL2, ...]

(최대 10개까지)

만약 적절한 코드 예시나 사용법, 오류 해결법, 설정 방법 등이 문서에 충분히 없다면 "생성할 수 없음"이라고만 응답하세요."""
                },
                {
                    "role": "user",
                    "content": f"다음 구글 api의 People api 관련 문서를 정확히 분석해서, 코드 예시, 사용법, 오류 해결법 등 실용적인 내용을 기반으로 최대 10개의 질문-답변과 해당 출처 URL들을 찾아주세요. 적절한 내용이 없다면 '생성할 수 없음'이라고 응답하세요:\n\n{text}"
                }
            ],
            max_tokens=2000,
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"질문-답변 및 출처 생성 중 오류 발생: {e}")
        return None


def parse_single_qa_block(block):
    """단일 질문-답변-출처 블록 파싱"""
    try:
        qa_dict = {}
        lines = block.strip().split('\n')

        current_section = None
        question_lines = []
        answer_lines = []
        source_lines = []

        for line in lines:
            line = line.strip()
            if not line:
                continue

            # 질문 시작 패턴
            if re.match(r'^질문\d*:', line):
                current_section = 'question'
                question_content = re.sub(r'^질문\d*:', '', line).strip()
                if question_content:
                    question_lines.append(question_content)

            # 답변 시작 패턴
            elif re.match(r'^답변\d*:', line):
                current_section = 'answer'
                answer_content = re.sub(r'^답변\d*:', '', line).strip()
                if answer_content:
                    answer_lines.append(answer_content)

            # 출처 시작 패턴
            elif re.match(r'^출처\d*:', line):
                current_section = 'sources'
                source_content = re.sub(r'^출처\d*:', '', line).strip()
                if source_content:
                    source_lines.append(source_content)

            # 연속되는 내용 라인
            else:
                if current_section == 'question':
                    question_lines.append(line)
                elif current_section == 'answer':
                    answer_lines.append(line)
                elif current_section == 'sources':
                    source_lines.append(line)

        # 각 섹션 조합
        if question_lines:
            qa_dict['question'] = ' '.join(question_lines).strip()

        if answer_lines:
            qa_dict['answer'] = ' '.join(answer_lines).strip()

        # URL은 그대로 사용 (중복 파싱 제거)
        if source_lines:
            # AI가 이미 주는 URL을 그대로 분리
            urls = []
            for s in source_lines:
                urls.extend([u.strip() for u in s.split() if u.startswith("http")])
            qa_dict['sources'] = urls if urls else ["출처를 찾을 수 없음"]
        else:
            qa_dict['sources'] = ["출처를 찾을 수 없음"]

        return qa_dict

    except Exception as e:
        print(f"단일 QA 블록 파싱 중 오류: {e}")
        return {}


# AI 응답을 파싱하는 함수 (개선된 버전)
def parse_qa_and_sources(ai_response):
    """개선된 질문-답변-출처 파싱 함수"""
    try:
        # "생성할 수 없음" 응답 체크
        if "생성할 수 없음" in ai_response:
            print("  -> 적절한 내용이 없어서 질문-답변을 생성하지 않음")
            return []

        qa_pairs = []

        # 전체 텍스트를 질문 단위로 분할
        # 질문1:, 질문2: 등의 패턴으로 분할
        question_blocks = re.split(r'\n(?=질문\d*:)', ai_response.strip())

        for block in question_blocks:
            if not block.strip():
                continue

            # 각 블록에서 질문, 답변, 출처 추출
            qa_dict = parse_single_qa_block(block)
            if qa_dict and qa_dict.get('question') and qa_dict.get('answer'):
                # 데이터 정리
                cleaned_qa = {
                    'question': qa_dict['question'].strip(),
                    'answer': qa_dict['answer'].strip(),
                    'sources': qa_dict.get('sources', ["출처를 찾을 수 없음"])
                }
                qa_pairs.append(cleaned_qa)

        print(f"  -> 파싱된 QA 쌍 개수: {len(qa_pairs)}")
        return qa_pairs

    except Exception as e:
        print(f"응답 파싱 중 오류: {e}")
        return []


# 메인 처리 함수
def process_files_and_generate_jsonl():
    jsonl_data = []

    for filename in os.listdir(files_dir):
        if filename.endswith('.txt'):
            file_path = os.path.join(files_dir, filename)
            print(f"처리 중: {filename}")

            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            # AI가 질문-답변과 출처를 한 번에 생성
            ai_response = generate_qa_and_sources(text)
            if ai_response:
                # AI 응답 파싱 (여러 개의 질문-답변 쌍)
                qa_pairs = parse_qa_and_sources(ai_response)

                if qa_pairs:
                    print(f"  -> {len(qa_pairs)}개의 질문-답변 쌍 생성됨")
                    for i, qa in enumerate(qa_pairs, 1):
                        print(f"    질문{i}: {qa['question'][:50]}...")
                        print(f"    출처{i}: {qa['sources']}")

                        # 메타데이터 설정
                        metadata = {
                            "question": qa['question'],
                            "answer": qa['answer'],
                            "sources": qa['sources'],
                            "tags": "people",
                            "last_verified": "2025-08-19",
                            "source_file": filename
                        }
                        jsonl_data.append(metadata)
                else:
                    print(f"  -> 적절한 내용이 없어서 건너뛰기: {filename}")
            else:
                print(f"  -> AI 응답 실패: {filename}")

    # JSONL 파일로 저장
    jsonl_filename = 'people_test_generated_qa.jsonl'
    with open(jsonl_filename, 'w', encoding='utf-8') as jsonl_file:
        for item in jsonl_data:
            jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f"\n완료! 총 {len(jsonl_data)}개의 질문-답변 쌍이 {jsonl_filename}에 저장되었습니다.")


# 실행
if __name__ == "__main__":
    process_files_and_generate_jsonl()


처리 중: people_quickstart_go.txt
  -> 파싱된 QA 쌍 개수: 10
  -> 10개의 질문-답변 쌍 생성됨
    질문1: People API를 사용하기 위해 필요한 기본 요건은 무엇인가요?...
    출처1: ['https://developers.google.com/people/quickstart/go?hl=ko#prerequisites']
    질문2: Google Cloud 프로젝트에서 People API를 사용 설정하는 방법은 무엇인가요?...
    출처2: ['https://developers.google.com/people/quickstart/go?hl=ko#enable_the_api']
    질문3: OAuth 동의 화면을 구성하는 방법은 무엇인가요?...
    출처3: ['https://developers.google.com/people/quickstart/go?hl=ko#configure_the_oauth_consent_screen']
    질문4: 데스크톱 애플리케이션의 사용자 인증 정보를 승인하는 방법은 무엇인가요?...
    출처4: ['https://developers.google.com/people/quickstart/go?hl=ko#authorize_credentials_for_a_desktop_application']
    질문5: People API를 호출하는 Go 애플리케이션의 샘플 코드는 어떻게 작성하나요?...
    출처5: ['https://developers.google.com/people/quickstart/go?hl=ko#set_up_the_sample']
    질문6: 샘플 애플리케이션을 실행하는 방법은 무엇인가요?...
    출처6: ['https://developers.google.com/people/quickstart/go?hl=ko#run_the_sample']
    질문7: People API를 통해 사용자의 연결 목록을 가져오는 방법은 무엇인가요?...
   

In [12]:
import os
import re
import json
import time
import hashlib
import datetime
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv

load_dotenv()

# =========================
# 설정
# =========================
ROOT_DIR = "./test"   # 재귀 순회할 최상위 폴더
OUT_JSONL = "./people_test_generated_qa_edit.jsonl"         # 결과를 저장할 JSONL 파일
MODEL = "gpt-4o-mini"
PAIR_MAX_QA = 5                          # 페어(또는 단일 청크)당 최대 Q&A 개수
CHUNK_TOKENS = 900                       # 청크 크기(토큰 기준)
CHUNK_OVERLAP_TOKENS = 150                # 청크 오버랩(토큰 기준)
PAIR_WINDOW = 2                          # 연속 청크 페어 크기
MAX_CONTEXT_TOKENS = 4096                # 모델 컨텍스트 상한
MAX_RETRY = 4                            # API 재시도 횟수

client = OpenAI()                         # OPENAI_API_KEY 필요
enc = tiktoken.get_encoding("cl100k_base")

# =========================
# 유틸
# =========================
def parse_source_meta(text):
    """문서 상단에서 Source URL 추출."""
    head = text[:2000]  # 문서 상단에서 첫 2000자만 처리
    m_url = re.search(r'(?i)^Source\s*URL\s*:\s*(\S+)', head, flags=re.M)
    return m_url.group(1).strip() if m_url else None

def smart_split(text):
    """토큰 기반 청킹 + 오버랩."""
    text = text.replace("\r\n", "\n").strip()
    if not text:
        return []
    toks = enc.encode(text)  # tiktoken으로 텍스트를 토큰으로 변환
    chunks = []
    step = max(1, CHUNK_TOKENS - CHUNK_OVERLAP_TOKENS)
    for i in range(0, len(toks), step):
        block = toks[i:i + CHUNK_TOKENS]  # 청크 크기만큼 자르기
        if not block: break
        chunk_text = enc.decode(block).strip()
        if chunk_text:
            chunks.append(chunk_text)
    return chunks

def make_pairs(chunks, window=PAIR_WINDOW):
    """연속 청크 페어 목록 생성: (0,[0,1]), (1,[1,2]), …"""
    if window < 2 or len(chunks) < 2:
        return []
    return [(i, chunks[i:i+window]) for i in range(len(chunks) - (window - 1))]

def trim_to_context_limit(text):
    """컨텍스트 초과 시 텍스트를 토큰 기준으로 상한 내로 자름."""
    toks = enc.encode(text)
    if len(toks) <= MAX_CONTEXT_TOKENS:
        return text
    return enc.decode(toks[:MAX_CONTEXT_TOKENS])

def hash_id(*parts):
    """안정적 레코드 ID 생성."""
    h = hashlib.sha256()
    for p in parts:
        h.update((p or "").encode("utf-8")); h.update(b"|")
    return h.hexdigest()[:32]

def json_loads_strict_or_strip_codefence(s):
    """
    response_format=json_object 덕분에 보통은 바로 json.loads로 충분.
    혹시 모를 코드펜스(```json ... ```)만 제거하는 얇은 래퍼.
    """
    s = s.strip()
    s = re.sub(r"^```(?:json)?\s*|\s*```$", "", s, flags=re.I|re.S)
    return json.loads(s)

# =========================
# 모델 호출
# =========================
def ask_model(pair_text, n, source_url):
    """
    해당 텍스트 범위에서 Q&A n개(JSON) 생성. 없으면 빈 리스트 반환.
    - 문서 범위 밖 정보 금지
    - 실무 친화적 질문/정확한 답변
    """
    if n <= 0:
        return []

    system_prompt = (
        "당신은 구글 API 중 PEOPLE API의 공식 문서 텍스트에서만 근거를 삼아 Q&A를 만듭니다. "
        "문서에 명시된 내용만 사용하고 추측은 금지합니다. 실무자가 바로 쓰도록 자세하고 이해하기 쉽게 답변해주세요."
    )
    url_hint = f"\n- 참고 URL(있으면): {source_url}" if source_url else ""
    user_prompt = f"""
아래는 한 문서의 (연속) 청크 범위입니다. 이 범위에서만 Q&A {n}개를 JSON으로 만들어 주세요.



**요구사항:**
- 문서 범위를 벗어난 정보 금지
- 질문은 실무 친화적으로 구체적·명확하게
- 답변은 문서 용어/표기 준수
- 각 항목: question, answer

**중요한 제약사항:**
- 문서에 명시된 내용만을 기반으로 질문과 답변을 작성하세요
- 문서에 없는 내용이나 추측, 일반적인 지식을 추가하지 마세요
- 답변은 반드시 문서 내용을 직접 참조해야 합니다
- 확실하지 않은 내용은 포함하지 마세요
- 만약 적절한 코드 예시나 사용법, 오류 해결법, 설정 방법 등이 문서에 충분히 없다면 "생성할 수 없음"이라고만 응답하세요.

**우선적으로 다룰 주제:**
1. 코드 예시와 구현 방법
2. API 사용법과 파라미터 설명
3. 오류 해결 방법과 문제 해결책
4. 설정 방법과 구성 옵션
5. 실제 사용 사례와 예제

{url_hint}

[원문 시작]
{pair_text}
[원문 끝]

JSON 스키마:
{{
  "items": [
    {{
      "question": "…",
      "answer": "…"
    }}
  ]
}}
""".strip()

    for attempt in range(1, MAX_RETRY + 1):
        try:
            resp = client.chat.completions.create(
                model=MODEL,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt},
                ],
                response_format={"type": "json_object"},  # JSON 강제
                temperature=0.,
                max_tokens=1100,
            )
            data = json_loads_strict_or_strip_codefence(resp.choices[0].message.content)
            items = data.get("items", []) if isinstance(data, dict) else []
            return items[:n]  # 과도 생성 시 컷
        except Exception:
            if attempt == MAX_RETRY:
                return []
            time.sleep(0.8 * attempt)  # 간단 백오프

# =========================
# 문서 처리
# =========================
def build_record(q, a, doc_path, pair_index,
                 chunk_indices, passage_window,
                 source_url):
    """RAG 친화 JSON 레코드."""
    rid = hash_id(doc_path, str(pair_index), q, a)  # 고유 레코드 ID 생성
    return {
        "question": q.strip(),  # 질문
        "answer": a.strip(),  # 답변
        "sources": [source_url or f"file://{doc_path}"],   # Source URL
        "tags": "people",                                  # 고정
        "last_verified": "2025-08-19",                      # 고정
        "source_file": os.path.basename(doc_path),          # 파일 이름
    }

def process_one_file(file_path, out_fh):
    """
    단일 문서 처리:
    - 상단 메타(Source URL) 추출 → 토큰 청킹
    - 청크=1이면 그 1개로 최대 5개 생성
    - 청크>=2면 (연속 페어)마다 최대 5개 생성
    - 생성 없으면 스킵, 결과는 JSONL append
    """
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:  # 파일 열기
        text = f.read()  # 파일 내용 읽기
    if not text.strip():  # 비어 있는 파일은 처리하지 않음
        return 0

    source_url = parse_source_meta(text)  # Source URL 추출
    chunks = smart_split(text)  # 텍스트를 청크로 나누기
    written = 0  # 작성된 Q&A 수 초기화

    # (A) 청크가 1개뿐 → 그 1개로 최대 5개 생성
    if len(chunks) == 1:
        pair_text = trim_to_context_limit(chunks[0])  # 청크 텍스트 크기 제한
        items = ask_model(pair_text, PAIR_MAX_QA, source_url)  # Q&A 생성
        for it in items:
            q, a = (it.get("question") or "").strip(), (it.get("answer") or "").strip()  # Q&A 추출
            if not q or not a: continue  # 질문과 답변이 없으면 건너뜀
            rec = build_record(q, a, file_path, 0, [0], pair_text, source_url)  # Q&A 레코드 생성
            out_fh.write(json.dumps(rec, ensure_ascii=False) + "\n")  # JSONL에 저장
            written += 1  # 작성된 Q&A 수 증가
        return written

    # (B) 청크가 2개 이상 → (1,2), (2,3)… 페어마다 최대 5개 생성
    pairs = make_pairs(chunks, window=PAIR_WINDOW)  # 청크 페어 생성
    if not pairs:  # 페어가 없다면 스킵
        return 0

    for (pair_idx, cg) in pairs:  # 각 페어에 대해
        pair_text = trim_to_context_limit("\n\n---\n\n".join(cg))  # 청크 페어 텍스트 크기 제한
        items = ask_model(pair_text, PAIR_MAX_QA, source_url)  # Q&A 생성
        if not items:  # Q&A가 없다면 스킵
            continue
        chunk_indices = list(range(pair_idx, pair_idx + len(cg)))  # 청크 인덱스 생성
        for it in items:
            q, a = (it.get("question") or "").strip(), (it.get("answer") or "").strip()  # Q&A 추출
            if not q or not a: continue  # 질문과 답변이 없으면 건너뜀
            rec = build_record(q, a, file_path, pair_idx, chunk_indices, pair_text, source_url)  # Q&A 레코드 생성
            out_fh.write(json.dumps(rec, ensure_ascii=False) + "\n")  # JSONL에 저장
            written += 1  # 작성된 Q&A 수 증가

    return written

# =========================
# 엔트리포인트
# =========================
def walk_and_generate():
    """폴더 재귀 순회 → 모든 문서를 처리 → 하나의 JSONL로 누적 저장."""
    os.makedirs(os.path.dirname(OUT_JSONL) or ".", exist_ok=True)  # 저장할 폴더 생성
    total_docs, total_qas = 0, 0  # 총 문서와 Q&A 수 초기화
    with open(OUT_JSONL, "a", encoding="utf-8") as out_fh:  # JSONL 파일 열기 (append 모드)
        for root, _, files in os.walk(ROOT_DIR):  # ROOT_DIR 내 모든 파일 재귀 순회
            for name in files:  # 각 파일에 대해
                if not name.lower().endswith(".txt"): continue  # .txt 파일만 처리
                path = os.path.join(root, name)  # 파일 경로 만들기
                print(f"[DOC] {path}")  # 파일 경로 출력
                cnt = process_one_file(path, out_fh)  # 파일 처리 후 Q&A 수 반환
                print(f"  -> {cnt} QAs")  # 처리된 Q&A 수 출력
                total_docs += 1  # 문서 수 증가
                total_qas += cnt  # Q&A 수 증가
    print(f"\n[DONE] docs={total_docs}, qas={total_qas}, out={OUT_JSONL}")  # 전체 처리 결과 출력


walk_and_generate()

[DOC] ./test\people_quickstart_go.txt
  -> 20 QAs

[DONE] docs=1, qas=20, out=./people_test_generated_qa_edit.jsonl


In [16]:

import json


client = OpenAI()

def generate_coverage_report(key_points, qa_set):
    # LLM 평가 프롬프트 생성
    prompt = "다음은 샘플 문서에 대한 핵심 포인트입니다. 각 포인트가 아래 QA셋에 얼마나 잘 반영되었는지 평가해주세요.\n\n"

    # 핵심 포인트 추가
    for idx, point in enumerate(key_points, 1):
        prompt += f"{idx}. {point}\n"

    # QA셋 추가
    prompt += "\n다음은 QA셋입니다:\n"
    for idx, qa in enumerate(qa_set, 1):
        prompt += f"{idx}. Q: {qa['question']} A: {qa['answer']}\n"

    prompt += "\n각 핵심 포인트가 QA셋에서 반영되었는지 평가해 주세요. 반영 여부를 '반영됨', '부분 반영됨', '누락됨'으로 알려주세요."

    # GPT 모델로 평가 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 사용할 모델
        messages=[{"role": "system", "content": "You are a helpful assistant."},  # 시스템 메시지
                  {"role": "user", "content": prompt}],  # 사용자 메시지 (핵심 포인트 및 QA셋)
        max_tokens=1000,
        temperature=0.
    )

    # 응답에서 결과 추출
    evaluation = response.choices[0].message.content.strip()

    return evaluation

def load_key_points_from_file(filename):
    # 저장된 핵심 포인트 텍스트 파일을 읽어서 반환
    with open(filename, 'r', encoding='utf-8') as file:
        key_points = file.read().strip().split('\n')
    return key_points

def load_qa_set_from_jsonl(filename):
    # JSONL 파일에서 QA셋을 로드
    qa_set = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            qa_set.append(json.loads(line))
    return qa_set

# 핵심 포인트 파일과 QA셋 파일 경로
key_points_file = "key_points_people_quickstart_go.txt"
qa_set_file = "people_test_generated_qa.jsonl"
qa_set_file_edit = "people_test_generated_qa_edit.jsonl"

# 핵심 포인트와 QA셋 불러오기
key_points = load_key_points_from_file(key_points_file)
qa_set = load_qa_set_from_jsonl(qa_set_file)
qa_set_edit = load_qa_set_from_jsonl(qa_set_file_edit)

# LLM을 사용한 평가
evaluation_result = generate_coverage_report(key_points, qa_set)
evaluation_result_edit = generate_coverage_report(key_points, qa_set_edit)

# 결과 출력
print("기본 전처리 LLM 평가 결과:\n", evaluation_result)
print("수정 전처리 LLM 평가 결과:\n", evaluation_result_edit)


기본 전처리 LLM 평가 결과:
 각 핵심 포인트가 QA셋에서 반영되었는지 평가한 결과는 다음과 같습니다:

1. **목표 설정**: 반영됨
   - QA셋의 질문들은 People API를 사용하는 방법에 대한 정보를 제공하고 있으며, 목표 설정의 내용이 잘 반영되어 있습니다.

2. **기본 요건**: 반영됨
   - QA셋의 첫 번째 질문에서 기본 요건이 명확하게 언급되어 있습니다.

3. **API 사용 설정**: 반영됨
   - QA셋의 두 번째 질문에서 Google Cloud 프로젝트에서 People API를 사용 설정하는 방법이 잘 설명되어 있습니다.

4. **OAuth 동의 화면 구성**: 반영됨
   - QA셋의 세 번째 질문에서 OAuth 동의 화면 구성 방법이 상세히 설명되어 있습니다.

5. **클라이언트 ID 생성**: 부분 반영됨
   - QA셋의 네 번째 질문에서 클라이언트 ID 생성 방법이 언급되지만, OAuth 2.0 클라이언트 ID 생성에 대한 구체적인 내용이 부족합니다.

6. **작업 디렉터리 설정**: 누락됨
   - QA셋에는 작업 디렉터리 설정에 대한 질문이 없습니다.

7. **샘플 코드 작성**: 반영됨
   - QA셋의 다섯 번째 질문에서 샘플 코드 작성 방법이 잘 설명되어 있습니다.

8. **샘플 실행**: 반영됨
   - QA셋의 여섯 번째 질문에서 샘플 애플리케이션 실행 방법이 명확하게 설명되어 있습니다.

9. **토큰 저장**: 누락됨
   - QA셋에는 토큰 저장에 대한 질문이 없습니다.

10. **문제 해결**: 반영됨
    - QA셋의 여덟 번째 질문에서 인증 및 승인 문제 해결 방법이 잘 설명되어 있습니다.

종합적으로 평가하면, 일부 핵심 포인트는 잘 반영되었고, 몇 가지는 부분적으로 반영되거나 누락되었습니다.
수정 전처리 LLM 평가 결과:
 각 핵심 포인트가 QA셋에서 반영되었는지 평가한 결과는 다음과 같습니다:

1. **목표 설정**: 반영됨
   - QA셋의 5번 질문이 이 목표를 

In [21]:
import json

client = OpenAI()

def generate_coverage_report(key_points, qa_set):
    # LLM 평가 프롬프트 생성
    prompt = "다음은 샘플 문서에 대한 핵심 포인트입니다. 각 포인트가 아래 QA셋에 얼마나 잘 반영되었는지 평가해주세요.\n\n"

    # 핵심 포인트 추가 (10개로 고정)
    for idx, point in enumerate(key_points, 1):
        prompt += f"{idx}. {point}\n"

    # QA셋 추가
    prompt += "\n다음은 QA셋입니다:\n"
    for idx, qa in enumerate(qa_set, 1):
        prompt += f"{idx}. Q: {qa['question']} A: {qa['answer']}\n"

    prompt += "\n각 핵심 포인트가 QA셋에서 반영되었는지 평가해 주세요. 반영 여부를 '반영됨', '부분반영', '누락됨'으로 알려주세요."

    # GPT 모델로 평가 요청
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # 사용할 모델
        messages=[{"role": "system", "content": "You are a helpful assistant."},  # 시스템 메시지
                  {"role": "user", "content": prompt}],  # 사용자 메시지 (핵심 포인트 및 QA셋)
        max_tokens=1000,
        temperature=0.
    )

    # 응답에서 결과 추출
    evaluation = response.choices[0].message.content.strip()

    return evaluation

def load_key_points_from_file(filename):
    # 저장된 핵심 포인트 텍스트 파일을 읽어서 반환
    with open(filename, 'r', encoding='utf-8') as file:
        key_points = file.read().strip().split('\n')

    # 공백 문자열이나 빈 항목을 제거하고 유효한 핵심 포인트만 반환
    key_points = [point for point in key_points if point.strip()]  # 공백 제거
    return key_points

def load_qa_set_from_jsonl(filename):
    # JSONL 파일에서 QA셋을 로드
    qa_set = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            qa_set.append(json.loads(line))
    return qa_set

def calculate_coverage_score(key_points, evaluation_result):
    """
    평가 결과에서 '반영됨', '부분 반영', '누락됨'을 기반으로 커버리지를 계산
    - 반영됨 = 1점
    - 부분 반영됨 = 0.5점
    - 누락됨 = 0점
    """
    score = 0
    total_points = len(key_points)  # 핵심 포인트의 개수는 key_points에서 얻음

    # 평가 결과에서 각 핵심 포인트에 대한 반영 상태를 확인하고 점수 계산
    for line in evaluation_result.split('\n'):
        if "반영됨" in line:
            score += 1
        elif "부분반영" in line:
            score += 0.5
        # "누락됨"은 점수를 더하지 않음

    coverage_rate = (score / total_points) * 100
    return coverage_rate, score, total_points

# 핵심 포인트 파일과 QA셋 파일 경로
key_points_file = "key_points_people_quickstart_go.txt"
qa_set_file = "people_test_generated_qa.jsonl"
qa_set_file_edit = "people_test_generated_qa_edit.jsonl"

# 핵심 포인트와 QA셋 불러오기
key_points = load_key_points_from_file(key_points_file)
qa_set = load_qa_set_from_jsonl(qa_set_file)
qa_set_edit = load_qa_set_from_jsonl(qa_set_file_edit)

# LLM을 사용한 평가
evaluation_result = generate_coverage_report(key_points, qa_set)
evaluation_result_edit = generate_coverage_report(key_points, qa_set_edit)

# 정량적 커버율 계산
coverage_rate, covered_points, total_points = calculate_coverage_score(key_points, evaluation_result)
coverage_rate_edit, covered_points_edit, total_points_edit = calculate_coverage_score(key_points, evaluation_result_edit)

# 결과 출력
print("기본 전처리 LLM 평가 결과:\n", evaluation_result)
print(f"기본 전처리 커버율: {coverage_rate:.2f}% ({covered_points}/{total_points} 포인트 반영됨)")

print("수정 전처리 LLM 평가 결과:\n", evaluation_result_edit)
print(f"수정 전처리 커버율: {coverage_rate_edit:.2f}% ({covered_points_edit}/{total_points_edit} 포인트 반영됨)")


기본 전처리 LLM 평가 결과:
 각 핵심 포인트가 QA셋에서 반영되었는지 평가한 결과는 다음과 같습니다:

1. **목표 설정**: 반영됨
   - QA셋의 질문과 답변이 Google Workspace API를 호출하는 Go 애플리케이션 설정에 대한 내용을 포함하고 있습니다.

2. **기본 요건**: 반영됨
   - QA셋의 1번 질문과 답변이 기본 요건을 정확히 반영하고 있습니다.

3. **API 사용 설정**: 반영됨
   - QA셋의 2번 질문과 답변이 Google Cloud 프로젝트에서 People API를 사용 설정하는 방법을 잘 설명하고 있습니다.

4. **OAuth 동의 화면 구성**: 반영됨
   - QA셋의 3번 질문과 답변이 OAuth 동의 화면 구성 방법을 상세히 설명하고 있습니다.

5. **클라이언트 ID 생성**: 부분반영
   - QA셋의 4번 질문과 답변이 클라이언트 ID 생성에 대한 정보를 포함하고 있지만, JSON 파일을 다운로드하여 작업 디렉터리에 저장하는 부분이 명확히 언급되지 않았습니다.

6. **작업 디렉터리 설정**: 누락됨
   - QA셋에는 작업 디렉터리 설정에 대한 질문과 답변이 없습니다.

7. **샘플 코드 작성**: 반영됨
   - QA셋의 5번 질문과 답변이 샘플 코드 작성에 대한 내용을 잘 반영하고 있습니다.

8. **샘플 실행**: 반영됨
   - QA셋의 6번 질문과 답변이 샘플 애플리케이션 실행 방법을 정확히 설명하고 있습니다.

9. **토큰 저장**: 누락됨
   - QA셋에는 인증 후 토큰 저장에 대한 질문과 답변이 없습니다.

10. **문제 해결**: 반영됨
    - QA셋의 8번 질문과 답변이 인증 및 승인 문제 해결 방법을 잘 설명하고 있습니다.

종합적으로 평가하면, 일부 핵심 포인트는 잘 반영되었으나, 몇 가지는 누락되거나 부분적으로만 반영되었습니다.
기본 전처리 커버율: 75.00% (7.5/10 포인트 반영됨)
수정 전처리 LLM 평가 결과:
 각 핵심 포인트가 QA셋